This file is now set for all 62 upstream gauges, end-to-end and available to run directly.

I run a test with several epochs, and the results are in the folder as well.

The first several lines make the codes connected to my personal Google's drive.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/My Drive/Benchmark'

In [ ]:
# This example is developed using tensorflow 1.15.
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import pandas as pd
    
def sequence_data_genearter():
	"""
  station_list = [519,521,522,523,525,526,527,532,534,535,536,537,538,539,541,542,
					543,546,549,551,552,553,554,556,557,562,564,565,568,569,572,574,
					590,595,599,601,604,605,607,616,617,621,624,626,631,640,641,642,
					644,648,653,655,656,659,661,662,663,665,668,669,670,671,540,544,
					563,566,570,571,573,575,579,585,586,587,588,589,591,592,593,594,
					596,597,598,600,602,603,606,608,609,610,611,612,1688,613,614,615,
					618,619,620,622,625,627,628,629,630,632,634,635,636,637,638,639,
					643,645,646,649,654,657,658,660,664,666,667,673,1609] # 125 staion IDs in IFIS
  """          
  # IN THIS TEST, my model can work on the following 62 watersheds.
	station_list = [553,542,522,536,569,543,538,568,574,535,671,539,546,617,534,653,554,552,551,616,527,607,525,565,644,661,562,656,665,642,641,526,556,599,659,670,621,668,648,662,601,624,541,532,557,669,523,537,521,590,549,572,640,631,519,595,605,655,663,564,604,626]


	# convert series to supervised learning sequence data
	def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
		n_vars = 1 if type(data) is list else data.shape[1]
		df = pd.DataFrame(data)
		cols, names = list(), list()
		# input sequence (t-n, ... t-1)
		for i in range(n_in, 0, -1):
			cols.append(df.shift(i))
			names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
		# forecast sequence (t, t+1, ... t+n)
		for i in range(0, n_out):
			cols.append(df.shift(-i))
			if i == 0:
				names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
			else:
				names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
		# put it all together
		agg = pd.concat(cols, axis=1)
		agg.columns = names
		# drop rows with NaN values
		if dropnan:
			agg.dropna(inplace=True)
		return agg
	
	stations_train = pd.DataFrame()
	stations_valid = pd.DataFrame()
	stations_test = pd.DataFrame()
	
	phy = pd.read_csv('./input/Features_Watershed.csv') # slope, travel time, area, and soil type data
	ET = pd.read_csv('./input/ET_Iowa.csv') # pretreated min-max scaled, hourly ET data
	
	for station_id in station_list:

		wq = pd.read_csv('./input/USGS/'+str(station_id)+'_wq.csv') # watershed discharge Q
		pcp = pd.read_csv('./input/PCP/'+str(station_id)+'_pcp_SMA6.csv') # hourly precipitation amount
		
		for datatype in ['train','valid','test']:
			if datatype == 'train':
				wq_station = wq[:35064] # the first 4 years as training
				pcp_station = pcp[:35064]
				ET_station = ET[:35064]
			elif datatype == 'valid':
				wq_station = wq[35064:52608] # the middle 2 years as validation
				pcp_station = pcp[35064:52608]
				ET_station = ET[35064:52608]
			elif datatype == 'test':
				wq_station = wq[52608:] # the last 1 year as test, the final evaluation
				pcp_station = pcp[52608:]
				ET_station = ET[52608:]
			
			dataset = pd.concat([wq_station, pcp_station, ET_station], axis=1) # combine the feature of discharge, rainfall, and ET.
			dataset = dataset.iloc[:,[3,4,1]] # feature shape = [?, 3], '?' is used to represent the instances as in TensorFlow
			dataset = dataset.fillna(-9999).values # fill NAs using negative values
			dataset = dataset.astype('float32') # convert to float
			
			# reframe the data as a supervised learning task
			reframed = series_to_supervised(dataset, hours_history, hours_forecast) # feature shape = [?, 576], (72+120)*3=576
			reframed_nanlist = reframed[(reframed < -1 ).any(1)].index.tolist() # find rows with any negative value (allowance of 1)
			reframed = reframed.drop(reframed_nanlist) # remove rows with negative value
			
			# combining physical data into reframed data
			phy_station = phy.loc[phy.ifc_id==station_id] # feature shape = [1, 16], the station name and 15 feature values
			phy_station2 = pd.DataFrame(pd.np.repeat(phy_station.values,reframed.shape[0],axis=0),columns=phy_station.columns) # feature shape = [?, 16]
			reframed = pd.concat([reframed.reset_index(drop=True), phy_station2.reset_index(drop=True)], axis=1, ignore_index=True) # feature shape = [?, 592]
			
			if datatype == 'train':
				stations_train = pd.concat([stations_train, reframed], axis=0)
			elif datatype == 'valid':
				stations_valid = pd.concat([stations_valid, reframed], axis=0)
			elif datatype == 'test':
				stations_test = pd.concat([stations_test, reframed], axis=0)

	return stations_train.values, stations_valid.values, stations_test.values

def min_max_normalization(dataset):

	# min-max normalization
	# the max precipitation and discharge should be calculated from the training+validation input.
  # Thus, in my generalized model on 62 watereshed. the maximum hourly rainfall in the training+validation period among all 62 watersheds is 899.9.
  # And, the maximum hourly discharge is 48775 in the training+validation period among all 62 watersheds.
  # If you train a different model, the values here should be different. And the same, the values for the de-normalization in the post-process should be different as well.
  
	PCPmax = 899.9092407226562
	PCPmin = 0.0
	Qmax = 48775.0
	Qmin = 0.0
	
	# normalize the Q and PCP
	PCPlist = list(range(0, dataset.shape[1], 3))[:hours_history+hours_forecast] # from column 0, every 3 features are PCP.
	Qlist = list(range(2, dataset.shape[1], 3))[:hours_history+hours_forecast] # from column 2, every 3 features are Q.
	dataset[:, PCPlist] = (dataset[:, PCPlist] - PCPmin) / (PCPmax-PCPmin)
	dataset[:, Qlist] = (dataset[:, Qlist] - Qmin) / (Qmax-Qmin)
	
	return dataset

def X_y_split(train, valid, test):

	# randomize the training data
	np.random.shuffle(train)
	
	# for the history 72 hours, we know PCP, ET, and Q, which is X1. shape = [?, 72, 3]
	# for the future 120 hours, we know PCP and ET, which is X2. shape = [?, 120, 2]
	# for each watershed, we know physical features, which is X3. shape = [?, 1, 15]
	# for the future 120 hours, we want to get Q, which is y. shape = [?, 120]
	
	# My method is generating two GRU layers, one for history and one for future.
	PCP_history = list(range(0, train.shape[1], 3))[:hours_history]
	PCP_forecast = list(range(0, train.shape[1], 3))[hours_history:hours_history+hours_forecast]
	ET_history = list(range(1, train.shape[1], 3))[:hours_history]
	ET_forecast = list(range(1, train.shape[1], 3))[hours_history:hours_history+hours_forecast]
	Q_history = list(range(2, train.shape[1], 3))[:hours_history]
	Q_forecast = list(range(2, train.shape[1], 3))[hours_history:hours_history+hours_forecast]
	
	X1_list = PCP_history + ET_history + Q_history
	X1_list.sort()
	X2_list = PCP_forecast + ET_forecast
	X2_list.sort()

	# split the data into X1, X2, X3 and y.
	train_X1, train_X2, train_X3, train_y, train_id = train[:,X1_list], train[:,X2_list], train[:,-15:], train[:,Q_forecast], train[:,-16:-15]
	valid_X1, valid_X2, valid_X3, valid_y, valid_id = valid[:,X1_list], valid[:,X2_list], valid[:,-15:], valid[:,Q_forecast], valid[:,-16:-15]
	test_X1, test_X2, test_X3, test_y, test_id = test[:,X1_list], test[:,X2_list], test[:,-15:], test[:,Q_forecast], test[:,-16:-15]

	# reshape X1 and X2 into 3D [samples, timesteps, features]
	train_X1 = train_X1.reshape(train_X1.shape[0], hours_history, 3)
	valid_X1 = valid_X1.reshape(valid_X1.shape[0], hours_history, 3)
	test_X1 = test_X1.reshape(test_X1.shape[0], hours_history, 3)
	train_X2 = train_X2.reshape(train_X2.shape[0], hours_forecast, 2)
	valid_X2 = valid_X2.reshape(valid_X2.shape[0], hours_forecast, 2)
	test_X2 = test_X2.reshape(test_X2.shape[0], hours_forecast, 2)
	
	# expand X3 to 3D, shape from [?, 15] to [?, 1, 15].
	train_X3, valid_X3, test_X3 = np.expand_dims(train_X3, axis=1), np.expand_dims(valid_X3, axis=1), np.expand_dims(test_X3, axis=1)
	return [train_X1, train_X2, train_X3, train_y, train_id, valid_X1, valid_X2, valid_X3, valid_y, valid_id, test_X1, test_X2, test_X3, test_y, test_id]


In [ ]:
# parameters
hours_forecast = 120 # 120 hours forecast is the current goal.
hours_history = 72 # 72 hours of history is enough for predicting the future.

train, valid, test = sequence_data_genearter()

train = min_max_normalization(train)
valid = min_max_normalization(valid)
test = min_max_normalization(test)

train_X1, train_X2, train_X3, train_y, train_id, valid_X1, valid_X2, valid_X3, valid_y, valid_id, test_X1, test_X2, test_X3, test_y, test_id = X_y_split(train, valid, test)
del train, valid, test


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Design of Model

In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, CuDNNGRU, Flatten, TimeDistributed, Lambda, concatenate
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

def NRM_generalized_basic():

	# design network
	dim_dense=[128, 64, 64, 32, 32]
	drop=0.2

	phy_input = Input(shape=(train_X3.shape[1],train_X3.shape[2]))
	phy_input_zeros = Lambda(lambda x: x * 0)(phy_input)
	
	encoder_phy = Lambda(lambda x: K.repeat_elements(x, hours_history, axis=1))(phy_input_zeros) # simple encoder without physical data
	decoder_phy = Lambda(lambda x: K.repeat_elements(x, hours_forecast, axis=1))(phy_input) # physical data is necessary in decoder phase

	encoder_input = Input(shape=(train_X1.shape[1],train_X1.shape[2]))
	encoder_input_phy = concatenate([encoder_input,encoder_phy],axis=-1)
	
	encoder_rnn1 = CuDNNGRU(32, return_state=True, return_sequences=True)
	encoder_output1, encoder_hc1 = encoder_rnn1(encoder_input_phy)
	encoder_rnn2 = CuDNNGRU(32, return_state=True, return_sequences=True)
	encoder_output2, encoder_hc2 = encoder_rnn2(encoder_output1)
	encoder_rnn3 = CuDNNGRU(32, return_state=True, return_sequences=True)
	encoder_output3, encoder_hc3 = encoder_rnn3(encoder_output2)
	encoder_rnn4 = CuDNNGRU(32, return_state=True, return_sequences=True)
	encoder_output4, encoder_hc4 = encoder_rnn4(encoder_output3)
	encoder_rnn5 = CuDNNGRU(32, return_state=True)
	encoder_output5, encoder_hc5 = encoder_rnn5(encoder_output4)
	
	decoder_input = Input(shape=(train_X2.shape[1],train_X2.shape[2]))
	decoder_input_phy = concatenate([decoder_input,decoder_phy],axis=-1)
	decoder_rnn1 = CuDNNGRU(32, return_sequences=True)
	decoder_rnn2 = CuDNNGRU(32, return_sequences=True)
	decoder_rnn3 = CuDNNGRU(32, return_sequences=True)
	decoder_rnn4 = CuDNNGRU(32, return_sequences=True)
	decoder_rnn5 = CuDNNGRU(32, return_sequences=True)
	x = decoder_rnn1(decoder_input_phy, initial_state=encoder_hc1)
	x = decoder_rnn2(x, initial_state=encoder_hc2)
	x = decoder_rnn3(x, initial_state=encoder_hc3)
	x = decoder_rnn4(x, initial_state=encoder_hc4)
	x = decoder_rnn5(x, initial_state=encoder_hc5)

	for dim in dim_dense:
		x = TimeDistributed(Dense(dim, activation='relu'))(x)
		x = TimeDistributed(Dropout(drop))(x)
	main_out = TimeDistributed(Dense(1, activation='relu'))(x)
	main_out = Flatten()(main_out)
	
	model = Model(inputs=[encoder_input, decoder_input, phy_input], outputs=main_out)
	model.summary()
	return model

Other settings of the model.


In [ ]:
model = NRM_generalized_basic()

testname = './NRM_generalized_basic'

# some technologies used in training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
								  patience=5, cooldown=200, min_lr=1e-8)
earlystoping = EarlyStopping(monitor='val_loss', min_delta=0,
											 patience=10, verbose=1, mode='auto')
checkpoint = ModelCheckpoint(testname+'.h5', monitor='val_loss', verbose=1,
							 save_best_only=True, save_weights_only=True, mode='min')

# setup the loss function and optimizer
# optimizer
RMSprop=keras.optimizers.RMSprop(lr=0.00003)

# loss function
# I used this customized loss function. MSE would work as well as a default method.
def nseloss(y_true, y_pred):
  return K.mean(K.sum((y_pred-y_true)**2,axis=0)/K.sum((y_true-K.mean(y_true))**2,axis=0))

model.compile(optimizer=RMSprop, loss=nseloss)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 15)]      0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1, 15)        0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 72, 3)]      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 72, 15)       0           lambda[0][0]                     
_____________

In [ ]:
# train the model with large batci size for 5 epochs to get a good initialization weights with dense layers freezed.
history = model.fit([train_X1, train_X2, train_X3], train_y, epochs=100, batch_size=64,
					validation_data=([valid_X1, valid_X2, valid_X3], valid_y), callbacks=[reduce_lr, earlystoping, checkpoint], verbose=1)

# save training loss into local file
loss_train = history.history['loss']
loss_valid = history.history['val_loss']
loss_train = pd.DataFrame({'TrainLoss':loss_train})
loss_valid = pd.DataFrame({'TestLoss':loss_valid})
loss_epoches = pd.concat([loss_train, loss_valid], axis=1)
loss_name = testname + '-loss.csv'
loss_epoches.to_csv(loss_name, index = True)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1375835 samples, validate on 766268 samples
Epoch 1/100
1375808/1375835 [============================>.] - ETA: 0s - loss: 0.6070
Epoch 00001: val_loss improved from inf to 0.36188, saving model to ./NRM_generalized_basic.h5
1375835/1375835 [==============================] - 1112s 808us/sample - loss: 0.6070 - val_loss: 0.3619
Epoch 2/100
1375744/1375835 [============================>.] - ETA: 0s - loss: 0.3444
Epoch 00002: val_loss improved from 0.36188 to 0.28124, saving model to ./NRM_generalized_basic.h5
1375835/1375835 [==============================] - 1081s 786us/sample - loss: 0.3444 - val_loss: 0.2812
Epoch 3/100
1375808/1375835 [============================>.] - ETA: 0s - loss: 0.3086
Epoch 00003: val_loss improved from 0.28124 to 0.26835, saving model to ./NRM_generalized_basic.h5
1375835/1375835 [==============================] - 1071s 778us/sample - loss: 0.3086 - val_los

Evaluation

I am using four statistics now. NSE, KGE, bias, and r (np.corrcoef).
The most popular one is NSE, a traditional one. The second popular metric is KGE, a new metric proposed in 2009.


In [ ]:
def nse(y_true, y_pred):
	return 1-np.sum((y_pred-y_true)**2)/np.sum((y_true-np.mean(y_true))**2)

def kge(y_true, y_pred):
	kge_r = np.corrcoef(y_true,y_pred)[1][0]
	kge_a = np.std(y_pred)/np.std(y_true)
	kge_b = np.mean(y_pred)/np.mean(y_true)
	return 1-np.sqrt((kge_r-1)**2+(kge_a-1)**2+(kge_b-1)**2)

def bias(y_true, y_pred):
	return np.sum(y_pred)/np.sum(y_true)-1

#model.load_weights('./OneModelFor62Upstream.h5') # a baseline model I trained on all 125 watersheds.
model.load_weights('./NRM_generalized_basic.h5') # a baseline model I trained on all 125 watersheds.

Q_predict = model.predict([test_X1, test_X2, test_X3])


# post-treatment
# I used the min-max scalling in the beginning, so I scale back.
# Thus, the output was 0-1 scaled, we need to rescaled back to the real streamflow rates in cfs.
# In addition, current unit is cfs, can convert to cms here if needed.


# for detail, see function min_max_normalization()
Q_predict_cfs = Q_predict*48775.0 
test_y_cfs = test_y*48775.0

In [ ]:
station_list = [553,542,522,536,569,543,538,568,574,535,671,539,546,617,534,653,554,552,551,616,527,607,525,565,644,661,562,656,665,642,641,526,556,599,659,670,621,668,648,662,601,624,541,532,557,669,523,537,521,590,549,572,640,631,519,595,605,655,663,564,604,626]

for station_id in station_list:
  # locate the index of the station
  station_idx = np.argwhere(test_id.flatten() == station_id).flatten()

  Q_predict_station = Q_predict_cfs[station_idx]
  Q_true_station = test_y_cfs[station_idx]

  # Save files for later analysis, optional
  # np.savetxt(str(station_id)+'_time_series_true.csv',Q_true_station, delimiter=',')
  # np.savetxt(str(station_id)+'_time_series_pred.csv',Q_predict_station, delimiter=',')

  NSE_test_eachHour = []
  r_test_eachHour = []
  bias_test_eachHour = []
  KGE_test_eachHour = []
  for x in range(hours_forecast):
    valuePred_test=Q_predict_station[:,x]
    valueTrue_test=Q_true_station[:,x]
    NSE_test_eachHour.append(nse(valueTrue_test,valuePred_test))
    r_test_eachHour.append(np.corrcoef(valueTrue_test,valuePred_test)[0][1])
    bias_test_eachHour.append(bias(valueTrue_test,valuePred_test))
    KGE_test_eachHour.append(kge(valueTrue_test,valuePred_test))

  NSE_test_eachHour=pd.DataFrame(NSE_test_eachHour)
  NSE_test_eachHour.columns = ['NSE']
  r_test_eachHour=pd.DataFrame(r_test_eachHour)
  r_test_eachHour.columns = ['r']
  bias_test_eachHour=pd.DataFrame(bias_test_eachHour)
  bias_test_eachHour.columns = ['bias']
  KGE_test_eachHour=pd.DataFrame(KGE_test_eachHour)
  KGE_test_eachHour.columns = ['KGE']

  evaluation_result = pd.concat([NSE_test_eachHour, KGE_test_eachHour, r_test_eachHour, bias_test_eachHour], axis=1)
  evaluation_name = testname+'-'+str(station_id)+'-evaluation.csv'
  evaluation_result.to_csv(evaluation_name, index = True)

Post evaluation...

These are basic evaluation for each watershed and each hour.
We can then, calculate the median of these watersheds among 120 prediction hours. (1*120 values, each value is the median of the 62 watersheds)


In [ ]:
# up
import numpy as np
import pandas as pd
from pandas import DataFrame, concat

eval_station_list = station_list

NSE_median = []
KGE_median = []
r_median = []
bias_median = []
for i in range(120):	
    NSEs = []
    KGEs = []
    rs = []
    biases = []
    for station_id in eval_station_list:
        a = pd.read_csv('./NRM_generalized_basic-'+str(station_id)+'-evaluation.csv')
        result_nse = a['NSE'][i]
        result_kge = a['KGE'][i]
        result_r = a['r'][i]
        result_bias = a['bias'][i]
        NSEs.append(result_nse)
        KGEs.append(result_kge)
        rs.append(result_r)
        biases.append(result_bias)
    NSE_median.append(np.median(NSEs))
    KGE_median.append(np.median(KGEs))
    r_median.append(np.median(rs))
    bias_median.append(np.median(biases))
print(NSE_median)